# 训练数据准备


In [2]:
## 特征集加载
import xarray as xr
ds_all = xr.open_mfdataset(
    "D:/xarray/aligned2/Feature_all/*.nc",
    combine  = "by_coords",
    chunks   = {"time":1, "lat":500, "lon":500},
    parallel = True
)
ds_all

<xarray.Dataset> Size: 146GB
Dimensions:        (lon: 43200, lat: 21600, time: 3)
Coordinates:
    spatial_ref    int32 4B 0
  * lon            (lon) float64 346kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * lat            (lat) float64 173kB 90.0 89.99 89.98 ... -89.98 -89.99 -90.0
    band           int32 4B 1
  * time           (time) datetime64[ns] 24B 2015-01-01 2018-01-01 2020-01-01
Data variables: (12/14)
    DEM            (lat, lon) float32 4GB dask.array<chunksize=(500, 500), meta=np.ndarray>
    GDPpc          (time, lat, lon) float32 11GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    GDPtot         (time, lat, lon) float32 11GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    GURdist        (time, lat, lon) float32 11GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    Population     (time, lat, lon) float32 11GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    Powerdist      (lat, lon) float32 4GB dask.array<chunksize=(500, 500), meta=np.ndarray>
    ...             ...
    Slope          (lat, lon) float32 4GB dask.array<chunksize=(500, 500), meta=np.ndarray>
    TertiaryRoad   (lat, lon) float64 7GB dask.array<chunksize=(500, 500), meta=np.ndarray>
    gdmp           (time, lat, lon) float64 22GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    rsds           (time, lat, lon) float32 11GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    tas            (time, lat, lon) float64 22GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    wind           (time, lat, lon) float32 11GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>

In [5]:
#撂荒特征加载
import os
import xarray as xr


# === 1. 设置参数 ===
out_dir = r'D:\xarray\merged_chunk'
ca_chunk_lat_min, ca_chunk_lat_max = 0,111
ca_chunk_lon_min, ca_chunk_lon_max = 0,111 

# === 2. 加载加利福尼亚地区的chunks ===
all_chunks = []
for ci in range(ca_chunk_lat_min, ca_chunk_lat_max + 1):
    for cj in range(ca_chunk_lon_min, ca_chunk_lon_max + 1):
        path = os.path.join(out_dir, f'chunk_{ci}_{cj}.nc')
        # Skip specific chunks
        if os.path.exists(path):
            all_chunks.append(path)
        
all_chunks


ds_abandon = xr.open_mfdataset(
    all_chunks
)

# 这里要注意，year当初计算错误，需要-1。对abandonment_year变量执行-1操作
ds_abandon['abandonment_year'] = ds_abandon['abandonment_year'] - 1

ds_abandon

<xarray.Dataset> Size: 87GB
Dimensions:               (lat: 15500, time: 31, lon: 40200)
Coordinates:
  * lat                   (lat) float64 124kB -55.83 -55.82 ... 73.32 73.33
  * time                  (time) datetime64[ns] 248B 1992-01-01 ... 2022-01-01
  * lon                   (lon) float64 322kB -163.3 -163.3 ... 180.0 180.0
Data variables:
    abandonment_year      (lat, lon) float32 2GB dask.array<chunksize=(500, 500), meta=np.ndarray>
    abandonment_duration  (lat, lon) float32 2GB dask.array<chunksize=(500, 500), meta=np.ndarray>
    recultivation         (lat, lon) float32 2GB dask.array<chunksize=(500, 500), meta=np.ndarray>
    current_abandonment   (lat, lon) float32 2GB dask.array<chunksize=(500, 500), meta=np.ndarray>
    landcover             (time, lat, lon) float32 77GB dask.array<chunksize=(16, 250, 250), meta=np.ndarray>

In [24]:
import pandas as pd
# 加载光伏站点
def load_pv_sites(csv2018: str, csv2020: str):
    """
    Load PV site CSVs and combine into a single DataFrame without duplicates.
    """
    df18 = pd.read_csv(csv2018)
    df18["year"] = 2018
    df20 = pd.read_csv(csv2020)
    df20["year"] = 2020
    df = pd.concat([df18, df20], ignore_index=True)
    df = df.drop_duplicates(subset=["lat", "lon", "year"])
    return df

pv_2018=r'D:\xarray\aligned2\pv\PV_2018_for_aligning.csv'
pv_2020=r'D:\xarray\aligned2\pv\PV_2020_for_aligning.csv'

df=load_pv_sites(pv_2018,pv_2020)

df

,unique_id,p_area,capacity_m,lon,lat,country,year,power
0,1,13592.361840,1.201704,21.615342,38.115039,GRC,2018,NaN
1,2,3411.622302,0.309810,6.379806,50.191844,DEU,2018,NaN
2,3,20889.852880,1.752301,-0.895182,52.038515,GBR,2018,NaN
3,4,4154.511624,0.397576,21.655906,38.108923,GRC,2018,NaN
4,5,4794.474966,0.440007,14.279998,50.517348,CZE,2018,NaN
...,...,...,...,...,...,...,...,...
103928,35268,109.944469,NaN,14.724244,63.142422,SWE,2020,0.000000
103929,35269,41371.089830,NaN,22.956362,62.845604,FIN,2020,2.757758
103930,35270,12767.612290,NaN,23.075502,60.858883,FIN,2020,1.024186
103931,35271,18.517843,NaN,30.371145,60.009571,RUS,2020,0.000000


In [25]:
df2020=pd.read_csv(pv_2020)
df2020


,unique_id,country,p_area,power,lat,lon,year
0,1,CAN,140.626685,0.000000,48.732666,-123.496376,2020
1,2,USA,140.626685,0.000000,47.020474,-120.223221,2020
2,3,USA,140.626685,0.000000,44.582127,-123.286964,2020
3,4,USA,140.626685,0.000000,37.405178,-122.084770,2020
4,5,USA,140.626685,0.000000,37.847172,-122.271530,2020
...,...,...,...,...,...,...,...
35267,35268,SWE,109.944469,0.000000,63.142422,14.724244,2020
35268,35269,FIN,41371.089830,2.757758,62.845604,22.956362,2020
35269,35270,FIN,12767.612290,1.024186,60.858883,23.075502,2020
35270,35271,RUS,18.517843,0.000000,60.009571,30.371145,2020


In [23]:
df2018 = pd.read_csv(pv_2018)
# Rename 'area' column to 'p_area'
if 'area' in df2018.columns:
    df2018 = df2018.rename(columns={'area': 'p_area'})
# Drop the 'pvout' column if it exists
if 'pvout' in df2018.columns:
    df2018 = df2018.drop(columns=['pvout'])
# Rename 'power' column to 'capacity_m' in df2020
if 'power' in df2020.columns:
    df2020 = df2020.rename(columns={'power': 'capacity_m'})
# Save the modified dataframe to file
df2018.to_csv(pv_2018.replace('.csv', '_for_aligning.csv'), index=False)
df2020.to_csv(pv_2020.replace('.csv', '_for_aligning.csv'), index=False)
df2020


,unique_id,country,p_area,capacity_m,lat,lon,year
0,1,CAN,140.626685,0.000000,48.732666,-123.496376,2020
1,2,USA,140.626685,0.000000,47.020474,-120.223221,2020
2,3,USA,140.626685,0.000000,44.582127,-123.286964,2020
3,4,USA,140.626685,0.000000,37.405178,-122.084770,2020
4,5,USA,140.626685,0.000000,37.847172,-122.271530,2020
...,...,...,...,...,...,...,...
35267,35268,SWE,109.944469,0.000000,63.142422,14.724244,2020
35268,35269,FIN,41371.089830,2.757758,62.845604,22.956362,2020
35269,35270,FIN,12767.612290,1.024186,60.858883,23.075502,2020
35270,35271,RUS,18.517843,0.000000,60.009571,30.371145,2020
